In [48]:
import requests
from bs4 import BeautifulSoup
import json
import math
import re
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

### Using JSON

JSON - JavaScript Object Notation, is currently becoming the most common form of data interchange. Python includes a json module for working with data in that format, and can easily ajust between the raw string of the JSON stream, and assorted python data types.

In [50]:
r = requests.get("https://www.statbureau.org/get-data-json?country=united-states")

In [51]:
inflation = json.loads(r.text)
type(inflation)

list

In [41]:
def get_adj_price(year, price):
    req_url = "https://www.statbureau.org/calculate-inflation-price-json?country=united-states&start={0!s}/1/1&end=2016/8/1&amount={1!s}"
    response = requests.get(req_url.format(year, price))
    if response.status_code == 200:
        adj_val = float(response.json().replace('$','').replace(',','').replace(' ', ''))
    else:
        adj_val= math.nan
    return adj_val

print(get_adj_price(1967, 3.50))

25.6


In [42]:
moviepage = requests.get("http://www.the-numbers.com/movie/records/All-Time-Domestic-Box-Office")
moviesoup = BeautifulSoup(moviepage.text, "lxml")

nondecimal = re.compile(r'[^\d]+')
yearlist = []
titlelist = []
domlist = []
interlist = []
worldlist = []

movierows = moviesoup.find_all('tr')
movierows = movierows[1:]

for row in movierows:
    cols = row.find_all('td')
    cols = cols[1:]
    
    yearlist.append(int(cols[0].text))
    titlelist.append(cols[1].text)
    domlist.append(int(nondecimal.sub('', cols[2].text)))
    interlist.append(int(nondecimal.sub('', cols[3].text)))
    worldlist.append(int(nondecimal.sub('', cols[4].text)))
    
movieDataDict = {}
movieDataDict['year'] = yearlist
movieDataDict['title'] = titlelist
movieDataDict['domestic'] = domlist
movieDataDict['international'] = interlist
movieDataDict['worldwide'] = worldlist

movieframe = pd.DataFrame(movieDataDict)
movieframe.head()

,domestic,international,title,worldwide,year
0,936662225,1122000000,Star Wars Ep. VII: The Force Awakens,2058662225,2015
1,760507625,2023411357,Avatar,2783918982,2009
2,658672302,1548943366,Titanic,2207615668,1997
3,652198010,1018130015,Jurassic World,1670328025,2015
4,623279547,896200000,The Avengers,1519479547,2012


In [62]:
def price_wrapper(the_row):
    return get_adj_price(the_row.year, the_row.domestic)

movieframe['adjusted domestic'] = movieframe.apply(price_wrapper, axis=1)
movieframe.head()

,domestic,international,title,worldwide,year,adjusted_domestic,adjusted domestic
0,936662225,1122000000,Star Wars Ep. VII: The Force Awakens,2058662225,2015,9.607597e+08,9.607597e+08
1,760507625,2023411357,Avatar,2783918982,2009,8.712947e+08,8.712947e+08
2,658672302,1548943366,Titanic,2207615668,1997,1.000272e+09,1.000272e+09
3,652198010,1018130015,Jurassic World,1670328025,2015,6.689771e+08,6.689771e+08
4,623279547,896200000,The Avengers,1519479547,2012,6.652077e+08,6.652077e+08


In [63]:
movieframe.sort_values(by=['adjusted domestic'], ascending=False).head(10)

,domestic,international,title,worldwide,year,adjusted_domestic,adjusted domestic
8,460998007,325600000,Star Wars Ep. IV: A New Hope,786598007,1977,1.907779e+09,1.907779e+09
89,260000000,210700000,Jaws,470700000,1975,1.206585e+09,1.206585e+09
12,435110554,357854772,ET: The Extra-Terrestrial,792965326,1982,1.114869e+09,1.114869e+09
2,658672302,1548943366,Titanic,2207615668,1997,1.000272e+09,1.000272e+09
0,936662225,1122000000,Star Wars Ep. VII: The Force Awakens,2058662225,2015,9.607597e+08,9.607597e+08
73,290271960,243900000,Star Wars Ep. V: The Empire Strikes Back,534171960,1980,9.115107e+08,9.115107e+08
1,760507625,2023411357,Avatar,2783918982,2009,8.712947e+08,8.712947e+08
56,309205079,263500000,Star Wars Ep. VI: Return of the Jedi,572705079,1983,7.630427e+08,7.630427e+08
15,422780140,564700000,The Lion King,987480140,1994,6.984079e+08,6.984079e+08
7,474544677,552500000,Star Wars Ep. I: The Phantom Menace,1027044677,1999,6.973490e+08,6.973490e+08


### Using a database API
For this, the OMDb API will be used, to pull additional data from their API based on a query string.
4. The data returned from the API will be in JSON format.
8. An appropriate column will be added to the pandas dataframe, and the additional data inserted.
2. The data will be resorted based on the rating from rotten tomatoes, for the sake of testing.

In [76]:
def get_rating(title):
    req_url = "http://www.omdbapi.com/?t={0!s}&tomatoes=true"
    response = requests.get(req_url.format(title))
    
    if response.status_code == 200:
        try:
            rating = response.json()['tomatoMeter']
            if rating == 'N/A':
                rating = math.nan
        except KeyError:
            rating = math.nan
    else:
        rating = math.nan
    
    return rating

print(get_rating("Titanic"))

88


In [77]:
def rating_wrapper(the_row):
    return get_rating(the_row.title)

movieframe['tomato rating'] = movieframe.apply(rating_wrapper, axis=1)
movieframe.head()

,domestic,international,title,worldwide,year,adjusted_domestic,adjusted domestic,tomato rating
0,936662225,1122000000,Star Wars Ep. VII: The Force Awakens,2058662225,2015,9.607597e+08,9.607597e+08,NaN
1,760507625,2023411357,Avatar,2783918982,2009,8.712947e+08,8.712947e+08,83
2,658672302,1548943366,Titanic,2207615668,1997,1.000272e+09,1.000272e+09,88
3,652198010,1018130015,Jurassic World,1670328025,2015,6.689771e+08,6.689771e+08,71
4,623279547,896200000,The Avengers,1519479547,2012,6.652077e+08,6.652077e+08,92


In [78]:
movieframe.sort_values(by=['tomato rating'], ascending=False).head(10)

,domestic,international,title,worldwide,year,adjusted_domestic,adjusted domestic,tomato rating
25,380529370,555900000,Finding Nemo,936429370,2003,5.066426e+08,5.066426e+08,99
16,415004880,654813349,Toy Story 3,1069818229,2010,4.628647e+08,4.628647e+08,99
39,341268248,680500000,Zootopia,1021768248,2016,3.475129e+08,3.475129e+08,98
34,356461711,497461374,Inside Out,853923085,2015,3.656324e+08,3.656324e+08,98
12,435110554,357854772,ET: The Extra-Terrestrial,792965326,1982,1.114869e+09,1.114869e+09,98
67,293004164,438538457,Up,731542621,2009,3.356876e+08,3.356876e+08,98
89,260000000,210700000,Jaws,470700000,1975,1.206585e+09,1.206585e+09,97
75,289423425,270334294,"Monsters, Inc.",559757719,2001,4.006236e+08,4.006236e+08,96
38,342548984,592154195,The Lord of the Rings: The Two Towers,934703179,2002,4.669154e+08,4.669154e+08,96
92,257784718,199944670,The Lego Movie,457729388,2014,2.664170e+08,2.664170e+08,96
